# 使用 FNO 及其变体的数据驱动达西流

在本笔记中，我们将介绍傅里叶神经算子及其一些变体背后的一些理论。然后，我们将以数据驱动的方式将这些架构应用于达西流场预测问题。

### 学习成果
1. 傅里叶神经算子 (FNO)、自适应傅里叶神经算子 (AFNO) 和物理信息神经算子 (PINO) 的简要理论
2. 如何在 Modulus 中设置和训练这三个模型
3. 三个变体之间的差异

## 傅里叶神经算子 (FNO)

傅里叶神经算子 (FNO) 是一种数据驱动的架构，可用于参数化偏微分方程解的分布。FNO 的关键特性是谱卷积：将积分核置于傅里叶空间的运算。谱卷积（傅里叶积分算子）定义如下：

\begin{equation}
(\mathcal{K}(\mathbf{w})\phi)(x) = \mathcal{F}^{-1}(R_{\mathbf{W}}\cdot \left(\mathcal{F}\right)\phi)(x), \quad \forall x \in D
\end{equation}

其中 $\mathcal{F}$ 和 $\mathcal{F}^{-1}$ 分别是傅里叶正变换和逆变换。
$R_{\mathbf{w}}$ 是包含可学习参数 $\mathbf{w}$ 的变换。注意，此算子是在整个*结构化欧几里得*域 $D$（离散化为 $n$ 个点）上计算的。

快速傅里叶变换 (FFT) 用于高效地执行傅里叶变换，所得变换 $R_{\mathbf{w}}$ 只是一个有限大小的可学习权重矩阵。在谱卷积内部，傅里叶系数被截断为仅包含较低阶模态，从而可以明确控制谱空间和线性算子的维数。

FNO 模型由一个全连接的“提升”层、带有逐点线性跳跃连接的 $L$ 个谱卷积以及位于末端的解码逐点全连接神经网络组成。

\begin{equation}
u_{net}(\Phi;\theta) = \mathcal{Q}\circ \sigma(W_{L} + \mathcal{K}_{L}) \circ ... \circ \sigma(W_{1} + \mathcal{K}_{1})\circ \mathcal{P}(\Phi), \quad \Phi=\left\{\phi(x); \forall x \in D\right\}
\end{equation}

...其中 $\sigma(W_{i} + \mathcal{K}_{i})$ 是谱卷积层 $i$，其逐点线性变换为 $W_{i}$，激活函数为 $\sigma(\cdot)$。 $\mathcal{P}$ 是逐点提升网络，它将输入投影到更高维的潜在空间，$\mathcal{P}:\mathbb{R}^{d_in} \rightarrow \mathbb{R}^{k}$。

类似地，$\mathcal{Q}$ 是逐点全连接解码网络，$\mathcal{P}:\mathbb{R}^{k} \rightarrow \mathbb{R}^{d_out}$。由于 FNO 的所有全连接组件都是逐点操作，因此该模型对输入的维数具有不变性。

<img src="fno_darcy.png" alt="Drawing" style="width: 900px;"/>

更多详情，请参阅 [Modulus 用户文档](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/user_guide/theory/architectures.html#fourier-neural-operator)。

## 自适应傅里叶神经算子 (AFNO)

与采用卷积架构的傅里叶神经算子相比，AFNO 利用了计算机视觉领域中最新的 Transformer 架构。视觉 Transformer 在计算机视觉领域取得了巨大的成功，这主要归功于其有效的自注意力机制。为了应对这一挑战，Guibas 等人提出了 [自适应傅里叶神经算子 (AFNO)](https://www.researchgate.net/publication/356601975_Adaptive_Fourier_Neural_Operators_Efficient_Token_Mixers_for_Transformers)，作为傅里叶域中一种高效的注意力机制。AFNO 基于算子学习的原理基础，使我们能够在傅里叶域中高效地将注意力机制构建为连续的全局卷积。为了应对视觉领域的挑战，例如图像中的不连续性和高分辨率输入，AFNO 提出了对 FNO 的原理性架构修改，从而提高了内存和计算效率。这包括在通道混合权重上施加块对角结构，自适应地在标记之间共享权重，以及通过软阈值和收缩稀疏化频率模式。

AFNO 模型通常包括以下步骤：
1. 将输入图像划分为规则网格，其中包含大小相等的 $h \times w$ 个块，每个块的大小为 $p\times p$。
2. 将块嵌入到大小为 $d$ 的标记中，嵌入维度生成大小为 $h \times w \times d$ 的标记张量 ($X_{h\times w \times d}$)。
3. 将标记传递到多层 Transformer 架构中，执行空间和通道混合。
4. 在所有 Transformer 层的末尾，使用线性解码器将特征张量转换回图像空间。

对于步骤 3 中的每一层，AFNO 架构执行以下操作：

首先将 token 张量变换到傅里叶域，公式如下：

\begin{equation}
z_{m,n} = [\mathrm{DFT}(X)]_{m,n},
\end{equation}

其中 $m,n$ 是 patch 位置的索引，DFT 表示二维离散傅里叶变换。然后，该模型在傅里叶域中应用标记权重，并通过软阈值和收缩操作来增强稀疏性，如下所示：

\begin{equation} 
\tilde{z}_{m,n} = S_{\lambda} ( \mathrm{MLP}(z_{m,n})),
\end{equation}

其中，$S_{\lambda}(x) = \mathrm{sign}(x) \max(|x| - \lambda, 0)$，稀疏性控制参数为 $\lambda$；$\mathrm{MLP(\cdot)}$ 是一个两层的多层感知器，其块对角权重矩阵在所有块之间共享。

ANFO 层中的最后一个操作是逆傅里叶变换，将其变换回块域并添加残差连接，如下所示：

\begin{equation}
y_{m,n} = [\mathrm{IDFT}(\tilde{Z})]_{m,n} + X_{m,n}.
\end{equation}

<img src="afno_darcy.png" alt="Drawing" style="width: 900px;"/>

AFNO 模型的完整数学描述超出了本材料的讨论范围，更多详细信息请参阅 [Modulus 用户文档](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/user_guide/theory/architectures.html#adaptive-fourier-neural-operator)。

## 物理信息驱动的神经算子

李宗宜等人在这篇[论文](https://arxiv.org/abs/2111.03794)中提出了物理信息驱动的神经算子 (PINO)。PINO 方法用于偏微分方程 (PDE) 系统的替代建模，有效地将傅里叶非零 (FNO) 的数据信息驱动的监督学习框架与物理信息驱动的学习框架相结合。PINO 将 PDE 损失 $\mathcal{L}_{pde}$ 添加到傅里叶神经算子中。由于 PDE 损失会限制解空间，因此这减少了训练替代模型所需的数据量。
PDE 损失还会对替代机器学习 (ML) 模型计算出的解施加物理约束，使其成为一种可验证、准确且可解释的机器学习替代建模工具，极具吸引力。

为了解释 PINO 背后的关键概念，我们假设一个稳态偏微分方程 (PDE) 系统。按照[论文](https://arxiv.org/abs/2111.03794)中使用的符号，我们考虑一个偏微分方程，其表达式为：

\begin{equation} 
\mathcal{P}(u, a) = 0 , \text{ in } D \subset \mathbb{R}^d,
\end{equation}

\begin{equation}
u = g ,  \text{ in } \partial D.
\end{equation}

其中，$\mathcal{P}$ 为偏微分算子，$a$ 为系数/参数，$u$ 为偏微分方程的解。

在 FNO 框架中，代理机器学习模型由解算子 $\mathcal{G}^\dagger_{\theta}$ 给出，它将系数空间 $a$ 中任意给定系数映射到解 $u$。 FNO 采用监督学习的方式进行训练，训练数据的形式为输入/输出对 $\lbrace a_j, u_j \rbrace_{j = 1}^N$。FNO 的训练损失函数为：对所有训练对 $\lbrace a_i, u_i, \rbrace_{i=1}^N$ 求和，即 $\mathcal{L}_{data}(\mathcal{G}_\theta) = \lVert u - \mathcal{G}_\theta(a) \rVert^2$。

在 PINO 框架中，解算子通过一个额外的 PDE 损失函数进行优化，该损失函数由参数/系数空间中适当支持分布的独立同分布样本 $a_j$ 计算得出。

一般来说，PDE 损失函数涉及计算 PDE 算子，而这又涉及计算傅里叶神经算子拟设的偏导数。通常，这并非易事。PINO 的关键创新在于提供了多种计算算子拟设偏导数的方法，即：

1. 使用有限差分法 (FDM) 进行数值微分。
2. 通过谱导数计算数值微分。
3. 基于一阶“精确”导数和二阶 FDM 导数的混合微分。

有关更多详细信息，请参阅 [Modulus 用户文档](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/user_guide/theory/architectures.html#physics-informed-neural-operator)。

现在，有了这个理论背景，让我们使用这三种方法来解决达西流问题。

## 问题描述

达西方程 (Darcy PDE) 是一个二阶椭圆型偏微分方程，其形式如下：

\begin{equation}
-\nabla \cdot \left(k(\textbf{x})\nabla u(\textbf{x})\right) = f(\textbf{x}), \quad \textbf{x} \in D,
\end{equation}

...其中 $u(\textbf{x})$ 为流动压力，$k(\textbf{x})$ 为渗透率场，$f(\cdot)$ 为强迫函数。达西流可以参数化各种系统，包括多孔介质中的流动、弹性材料和热传导。这里，我们将定义域为一个二维单位正方形 $D=\left\{x,y \in (0,1)\right\}$，边界条件为 $u(\textbf{x})=0, \textbf{x}\in\partial D$。渗透率场和流场均离散为一个二维矩阵 $\textbf{K}, \textbf{U} \in \mathbb{R}^{N \times N}$。

本题的目标是建立一个替代模型，学习渗透率场和压力场 $\textbf{K} \rightarrow \textbf{U}$ 之间的映射，以获得渗透率场分布 $\textbf{K} \sim p(\textbf{K})$。
在本题中，你学习的不仅仅是一个解，而是一个分布。

## 下载所需数据

在开始任何编码之前，我们需要确保拥有训练数据和验证数据。本示例的训练数据集和验证数据集可以在 [Fourier Neural Operator GitHub 页面](https://github.com/zongyi-li/fourier_neural_operator) 上找到。脚本 [`utilities.py`](../../source_code/darcy/utilities.py) 是一个用于下载和转换此数据集的自动脚本。这需要使用软件包 [gdown](https://github.com/wkentaro/gdown)，可以通过 `pip install gdown` 轻松安装。

In [ ]:
import os
os.chdir('../../source_code/darcy/')

from utilities import download_FNO_dataset, load_FNO_dataset
from modulus.dataset import HDF5GridDataset
from modulus.key import Key

download_FNO_dataset("Darcy_241", outdir="datasets/")

## Part 1: 使用 FNO 求解

让我们看看如何使用 FNO 求解这个问题。完整的问题设置可以在 [`darcy_FNO_lazy.py`](../../source_code/darcy/darcy_FNO_lazy.py) 脚本中找到，但为了便于理解，我们将逐步构建约束和案例。[`config_FNO.yaml`](../../source_code/darcy/conf/config_FNO.yaml) 列出了该问题所需的配置。让我们先加载它们：

In [ ]:
from modulus.hydra import to_yaml, to_absolute_path
from modulus.hydra.utils import compose
from modulus.hydra.config import ModulusConfig

cfg = compose(config_path="../../source_code/darcy/conf", config_name="config_FNO")
cfg.network_dir = 'outputs/darcy_FNO'
print(to_yaml(cfg))

### 加载数据

对于这个数据驱动的问题，第一步是将训练数据导入 Modulus。在加载数据之前，我们可以设置任何想要应用于数据的归一化值。对于这个数据集，我们计算了输入渗透率场和输出压力的缩放和平移参数。然后，我们在 Modulus 中设置此归一化，方法是为每个键设置缩放/平移，即 `Key(name, scale=(shift, scale))`。

In [ ]:
input_keys = [Key("coeff", scale=(7.48360e00, 4.49996e00))]
output_keys = [Key("sol", scale=(5.74634e-03, 3.88433e-03))]

加载数据有两种方法。第一种是使用即时加载，即一次性将整个数据集读入内存。另一种方法是使用延迟加载，即根据模型训练所需的每个示例加载数据。即时加载可以消除训练期间从磁盘读取数据的潜在开销，但这种方法无法扩展到大型数据集。本例中，训练数据集和测试数据集都使用了延迟加载，以演示延迟加载在处理大型问题时的实用性。

In [ ]:
train_path = to_absolute_path("datasets/Darcy_241/piececonst_r241_N1024_smooth1.hdf5")
test_path = to_absolute_path("datasets/Darcy_241/piececonst_r241_N1024_smooth2.hdf5")

# make datasets
train_dataset = HDF5GridDataset(train_path, invar_keys=["coeff"], outvar_keys=["sol"], n_examples=1000)
test_dataset = HDF5GridDataset(test_path, invar_keys=["coeff"], outvar_keys=["sol"], n_examples=100)

如果您有兴趣了解预加载的工作原理，请参阅[`darcy_FNO.py`](../../source_code/darcy/darcy_FNO.py)脚本。

### 初始化模型和域，并添加约束

初始化模型和域的步骤与我们之前看到的其他 PINN 模型相同。

对于 Modulus 中的物理信息问题，我们通常需要根据边界条件和控制方程定义几何结构和约束。这里唯一的约束是 [`SupervisedGridConstraint`](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/api/modulus.domain.constraint.html#modulus.domain.constraint.discrete.SupervisedGridConstraint)，它对网格数据执行标准的监督训练。此约束支持使用多个 Worker，这在使用延迟加载时尤为重要。让我们看看如何在代码中实现这一点：

In [ ]:
from modulus.hydra import instantiate_arch
from modulus.domain import Domain
from modulus.domain.constraint import SupervisedGridConstraint

# make list of nodes to unroll graph on
model = instantiate_arch(
    input_keys=input_keys,
    output_keys=output_keys,
    cfg=cfg.arch.fno,
)
nodes = model.make_nodes(name="FNO", jit=cfg.jit)

# make domain
domain = Domain()

# add constraints to domain
supervised = SupervisedGridConstraint(
    nodes=nodes,
    dataset=train_dataset,
    batch_size=cfg.batch_size.grid,
    num_workers=4,  # number of parallel data loaders
)
domain.add_constraint(supervised, "supervised")

**注意**：网格数据是指可以像图像一样在张量中定义的数据。在 Modulus 中，此类数据网格通常表示一个空间域，并且应遵循 `[batch, channel, xdim, ydim, zdim]` 的标准维度，其中 channel 是状态变量的维度。傅里叶变换和卷积模型都使用基于网格的数据，以便在一次前向传播中高效地学习和预测整个域，这与标准 PINN 方法的逐点预测不同。

### 添加数据验证器

然后，我们可以使用 [`GridValidator`](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/api/modulus.domain.validator.html#modulus.domain.validator.discrete.GridValidator) 将验证数据添加到域中。该验证器适用于处理结构化数据。

In [ ]:
from modulus.domain.validator import GridValidator
from modulus.utils.io.plotter import GridValidatorPlotter

# add validator
val = GridValidator(
    nodes,
    dataset=test_dataset,
    batch_size=cfg.batch_size.validation,
    plotter=GridValidatorPlotter(n_examples=5),
)
domain.add_validator(val, "test")

### 求解器和模型训练

我们可以使用刚刚创建的域以及定义优化器选项的其他配置，并使用 Modulus 的 `Solver` 类设置来创建一个求解器。然后可以使用 `solve` 方法执行该求解器。

In [ ]:
# to make the logging work in the jupyter cells
# Need to execute this only once!
import logging
logging.getLogger().addHandler(logging.StreamHandler())

In [ ]:
import os
from modulus.solver import Solver

# optional set appropriate GPU in case of multi-GPU machine
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="2"

# make solver
slv = Solver(cfg, domain)

# start solver
slv.solve()

### 结果与后处理

检查点目录根据其导数的 `rec_results_freq` 参数中指定的结果记录频率保存。网络目录文件夹包含多个不同验证预测的图表。下图展示了几个图表，您可以看到该模型能够准确预测之前未见过的渗透率场的压力场。这些可视化效果也可以在训练过程中使用 Tensorboard 查看。

<figure>
    <figcaption>FNO 验证预测。从左到右为输入渗透率、真实压力、预测压力、误差。</figcaption>
    <img src="fno_darcy_pred1.png" alt="Drawing" style="width: 900px;"/>
    <img src="fno_darcy_pred2.png" alt="Drawing" style="width: 900px;"/>
    <img src="fno_darcy_pred3.png" alt="Drawing" style="width: 900px;"/>
</figure>

## Part 2: 使用 AFNO 求解

让我们看看如何使用 AFNO 求解这个问题。完整的问题设置可以在 [`darcy_AFNO.py`](../../source_code/darcy/darcy_AFNO.py) 脚本中找到，但为了便于理解，我们将逐步构建约束和案例。[`config_AFNO.yaml`](../../source_code/darcy/conf/config_AFNO.yaml) 列出了该问题所需的配置。让我们先加载它们。从案例设置的角度来看，设置 AFNO 训练的过程非常相似，因此我们只会在必要时强调重要的差异。

AFNO 基于 ViT Transformer 架构，需要对输入进行标记化。这里，每个标记都是图像的一个块，其块大小通过参数 `patch_size` 在配置文件中定义。`embed_dim` 参数定义了模型内部为每个块使用的潜在嵌入特征的大小。

In [ ]:
cfg = compose(config_path="../../source_code/darcy/conf", config_name="config_AFNO")
cfg.network_dir = 'outputs/darcy_AFNO'
print(to_yaml(cfg))

### 加载数据

将训练数据集和验证数据集加载到内存中的过程与 FNO 类似。AFNO 的输入需要能够被指定的块大小（在本例中为 `patch_size=16`）完全整除，而本数据集并非如此。因此，我们修剪输入/输出特征，使其具有合适的维度 `241x241 -> 240x240`。

In [ ]:
from modulus.dataset import DictGridDataset

# load training/ test data
input_keys = [Key("coeff", scale=(7.48360e00, 4.49996e00))]
output_keys = [Key("sol", scale=(5.74634e-03, 3.88433e-03))]

invar_train, outvar_train = load_FNO_dataset(
    "datasets/Darcy_241/piececonst_r241_N1024_smooth1.hdf5",
    [k.name for k in input_keys],
    [k.name for k in output_keys],
    n_examples=1000,
)
invar_test, outvar_test = load_FNO_dataset(
    "datasets/Darcy_241/piececonst_r241_N1024_smooth2.hdf5",
    [k.name for k in input_keys],
    [k.name for k in output_keys],
    n_examples=100,
)

# get training image shape
img_shape = [
    next(iter(invar_train.values())).shape[-2],
    next(iter(invar_train.values())).shape[-1],
]

# crop out some pixels so that img_shape is divisible by patch_size of AFNO
img_shape = [s - s % cfg.arch.afno.patch_size for s in img_shape]
print(f"cropped img_shape: {img_shape}")
for d in (invar_train, outvar_train, invar_test, outvar_test):
    for k in d:
        d[k] = d[k][:, :, : img_shape[0], : img_shape[1]]
        print(f"{k}: {d[k].shape}")

# make datasets
train_dataset = DictGridDataset(invar_train, outvar_train)
test_dataset = DictGridDataset(invar_test, outvar_test)

### 初始化模型和域，添加约束和验证器。

这些步骤与我们在 FNO 案例中看到的步骤相同。对于 AFNO，我们在加载数据集后计算域的大小。域需要在 AFNO 模型中定义，该模型通过在 `instantiate_arch` 调用中添加关键字参数 `img_shape` 来提供。

In [ ]:
# make list of nodes to unroll graph on
model = instantiate_arch(
    input_keys=input_keys,
    output_keys=output_keys,
    cfg=cfg.arch.afno,
    img_shape=img_shape,
)
nodes = [model.make_node(name="AFNO", jit=cfg.jit)]

# make domain
domain = Domain()

# add constraints to domain
supervised = SupervisedGridConstraint(
    nodes=nodes,
    dataset=train_dataset,
    batch_size=cfg.batch_size.grid,
)
domain.add_constraint(supervised, "supervised")

# add validator
val = GridValidator(
    nodes,
    dataset=test_dataset,
    batch_size=cfg.batch_size.validation,
    plotter=GridValidatorPlotter(n_examples=5),
)
domain.add_validator(val, "test")

### 求解器和模型训练

设置好域和配置后，即可定义求解器，并像之前一样开始训练。

In [ ]:
# make solver
slv = Solver(cfg, domain)

# start solver
slv.solve()

### 结果与后处理

检查点目录根据其导数的 `rec_results_freq` 参数中指定的结果记录频率保存。网络目录文件夹包含不同验证预测的多个图表，其中一些如下所示。

<figure>
    <figcaption>AFNO 验证预测。从左到右为输入渗透率、真实压力、预测压力、误差。</figcaption>
    <img src="afno_darcy_pred1.png" alt="Drawing" style="width: 900px;"/>
    <img src="afno_darcy_pred2.png" alt="Drawing" style="width: 900px;"/>
    <img src="afno_darcy_pred3.png" alt="Drawing" style="width: 900px;"/>
</figure>

需要注意的是，AFNO 的优势在于它能够扩展到比本笔记本/示例中使用的更大的模型大小和数据集。虽然此处未进行说明，但本示例演示了如何在 Modulus 中使用 AFNO 架构进行数据驱动训练的基本实现，以便您将其扩展到更大的问题。

## Part 3: 使用 PINO 求解

PINO 与 FNO 的关键区别在于，PINO 在 FNO 的损失函数中添加了一个基于物理的项。正如理论中进一步讨论的那样，PINO 损失函数的描述如下：

\begin{equation}
\mathcal{L} = \mathcal{L}_{data} + \mathcal{L}_{pde},
\end{equation}

其中，
\begin{equation}
\mathcal{L}_{data} = \lVert u - \mathcal{G}_\theta(a)  \rVert^2 ,
\end{equation}

...其中 $\mathcal{G}_\theta(a)$ 是一个具有可学习参数 $\theta$ 和输入场 $a$ 的 FNO 模型，$\mathcal{L}_{pde}$ 是一个合适的 PDE 损失函数。对于二维达西问题，其公式如下：

\begin{equation}
\mathcal{L}_{pde} = \lVert -\nabla \cdot \left(k(\textbf{x})\nabla \mathcal{G}_\theta(a)(\textbf{x})\right) - f(\textbf{x}) \rVert^2 ,
\end{equation}

...其中 $k(\textbf{x})$ 为渗透率场，$f(\textbf{x})$ 为力函数，在本例中等于 1，$a=k$。

需要注意的是，PDE 损失函数涉及计算 FNO 拟设 $\mathcal{G}_\theta(a)$ 的各种偏导数。
Modulus 提供了三种不同的计算方法。这些方法基于 PINO 的原始论文：

1. 通过有限差分法 (FDM) 计算数值微分
2. 通过谱导数计算数值微分
3. 基于一阶“精确”导数和二阶 FDM 导数组合的混合微分

前两种方法与原始论文中提出的相同。第三种方法是对论文中提出的“精确”方法的修改。
由于该方法需要计算 Hessian 矩阵，因此在计算二阶导数时，该方法比数值导数方法速度更慢且占用更多内存。
因此，本文提出了一种“混合”方法，该方法通过将一阶“精确”（精确方法在技术上并不精确，因为它结合了数值谱导数和精确微分，更多详情请参阅原始论文）导数和二阶 FDM 导数相结合，提供了一种折衷方案。

现在考虑问题设置，它与 FNO 示例大致相同，只是定义了 PDE 损失函数，并用它来约束 FNO 模型。定义 PDE 损失函数时会详细描述此过程。

我们先加载此问题的配置。此问题的配置与 FNO 示例类似，但重要的是，它有一个额外的参数 `custom.gradient_method`，用于选择 PDE 损失函数中梯度的计算方法。该方法可以是 `fdm`、`fourier` 或 `hybrid`，分别对应上述三个选项。损失函数中数据项和 PDE 项之间的平衡也可以使用 `loss.weights` 参数组来控制。[`config_PINO.yaml`](../../source_code/darcy/conf/config_PINO.yaml) 已加载至下方。

In [ ]:
cfg = compose(config_path="../../source_code/darcy/conf", config_name="config_PINO")
cfg.network_dir = 'outputs/darcy_PINO'
print(to_yaml(cfg))

### 为网格数据定义 PDE 损失函数

在本例中，我们利用上面提出的各种方法定义了一个自定义的 PDE 残差计算方法。使用 SymPy 和自动微分函数定义自定义 PDE 残差的过程已在 [PINN 笔记本](../diffusion_1d/Diffusion_Problem_Notebook.ipynb) 中进行了讨论。对于这个问题，我们将不依赖标准的自动微分函数来计算导数，而是使用一个名为 `Darcy` 的自定义 `torch.nn.Module` 来明确定义残差的计算方法。该模块的目的是根据 FNO 模型的输入和输出张量，计算并返回 Darcy PDE 残差，这通过其 `.forward(...)` 方法完成。

In [ ]:
import torch
import torch.nn.functional as F
from typing import Dict

# import the custom ops
from ops import dx, ddx

class Darcy(torch.nn.Module):
    "Custom Darcy PDE definition for PINO"

    def __init__(self, gradient_method: str = "hybrid"):
        super().__init__()
        self.gradient_method = str(gradient_method)

    def forward(self, input_var: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # get inputs
        u = input_var["sol"]
        c = input_var["coeff"]
        dcdx = input_var["Kcoeff_y"]  # data is reversed
        dcdy = input_var["Kcoeff_x"]

        dxf = 1.0 / u.shape[-2]
        dyf = 1.0 / u.shape[-1]
        # Compute gradients based on method
        # Exact first order and FDM second order
        if self.gradient_method == "hybrid":
            dudx_exact = input_var["sol__x"]
            dudy_exact = input_var["sol__y"]
            dduddx_fdm = ddx(
                u, dx=dxf, channel=0, dim=0, order=1, padding="replication"
            )
            dduddy_fdm = ddx(
                u, dx=dyf, channel=0, dim=1, order=1, padding="replication"
            )
            # compute darcy equation
            darcy = (
                1.0
                + (dcdx * dudx_exact)
                + (c * dduddx_fdm)
                + (dcdy * dudy_exact)
                + (c * dduddy_fdm)
            )
        # FDM gradients
        elif self.gradient_method == "fdm":
            dudx_fdm = dx(u, dx=dxf, channel=0, dim=0, order=1, padding="replication")
            dudy_fdm = dx(u, dx=dyf, channel=0, dim=1, order=1, padding="replication")
            dduddx_fdm = ddx(
                u, dx=dxf, channel=0, dim=0, order=1, padding="replication"
            )
            dduddy_fdm = ddx(
                u, dx=dyf, channel=0, dim=1, order=1, padding="replication"
            )
            # compute darcy equation
            darcy = (
                1.0
                + (dcdx * dudx_fdm)
                + (c * dduddx_fdm)
                + (dcdy * dudy_fdm)
                + (c * dduddy_fdm)
            )
        # Fourier derivative
        elif self.gradient_method == "fourier":
            dim_u_x = u.shape[2]
            dim_u_y = u.shape[3]
            u = F.pad(
                u, (0, dim_u_y - 1, 0, dim_u_x - 1), mode="reflect"
            )  # Constant seems to give best results
            f_du, f_ddu = fourier_derivatives(u, [2.0, 2.0])
            dudx_fourier = f_du[:, 0:1, :dim_u_x, :dim_u_y]
            dudy_fourier = f_du[:, 1:2, :dim_u_x, :dim_u_y]
            dduddx_fourier = f_ddu[:, 0:1, :dim_u_x, :dim_u_y]
            dduddy_fourier = f_ddu[:, 1:2, :dim_u_x, :dim_u_y]
            # compute darcy equation
            darcy = (
                1.0
                + (dcdx * dudx_fourier)
                + (c * dduddx_fourier)
                + (dcdy * dudy_fourier)
                + (c * dduddy_fourier)
            )
        else:
            raise ValueError(f"Derivative method {self.gradient_method} not supported.")

        # Zero outer boundary
        darcy = F.pad(darcy[:, :, 2:-2, 2:-2], [2, 2, 2, 2], "constant", 0)
        # Return darcy
        output_var = {
            "darcy": dxf * darcy,
        }  # weight boundary loss higher
        return output_var

FNO 解的梯度是根据上面选择的梯度法计算的。使用 `hybrid` 方法时，FNO 模型会自动输出一阶梯度，因此无需额外计算。此外，请注意，渗透率场的梯度已作为张量包含在 FNO 输入训练数据中（使用键 `Kcoeff_x` 和 `Kcoeff_y` ），因此无需额外计算。

### 加载数据并初始化模型

数据加载过程与 FNO 示例类似。仅在使用混合梯度法的情况下，您需要额外指示模型输出适当的梯度，方法是在其输出键中指定这些梯度。

我们将之前定义的 `Darcy` 模型包装到 Modulus 的 `Node` 中，将其合并到 Modulus 中。这确保该模块被整合到 Modulus 的计算图中，并可用于优化 FNO。

In [ ]:
import numpy as np
from modulus.dataset import DictGridDataset
from modulus.node import Node

# load training/ test data
input_keys = [
    Key("coeff", scale=(7.48360e00, 4.49996e00)),
    Key("Kcoeff_x"),
    Key("Kcoeff_y"),
]
output_keys = [
    Key("sol", scale=(5.74634e-03, 3.88433e-03)),
]

invar_train, outvar_train = load_FNO_dataset(
    "datasets/Darcy_241/piececonst_r241_N1024_smooth1.hdf5",
    [k.name for k in input_keys],
    [k.name for k in output_keys],
    n_examples=cfg.custom.ntrain,
)
invar_test, outvar_test = load_FNO_dataset(
    "datasets/Darcy_241/piececonst_r241_N1024_smooth2.hdf5",
    [k.name for k in input_keys],
    [k.name for k in output_keys],
    n_examples=cfg.custom.ntest,
)

# Define FNO model
if cfg.custom.gradient_method == "hybrid":
    output_keys += [
        Key("sol", derivatives=[Key("x")]),
        Key("sol", derivatives=[Key("y")]),
    ]
model = instantiate_arch(
    input_keys=[input_keys[0]],
    output_keys=output_keys,
    cfg=cfg.arch.fno,
    domain_length=[1.0, 1.0],
)

# Make custom Darcy residual node for PINO
inputs = [
    "sol",
    "coeff",
    "Kcoeff_x",
    "Kcoeff_y",
]
if cfg.custom.gradient_method == "hybrid":
    inputs += [
        "sol__x",
        "sol__y",
    ]
darcy_node = Node(
    inputs=inputs,
    outputs=["darcy"],
    evaluate=Darcy(gradient_method=cfg.custom.gradient_method),
    name="Darcy Node",
)
nodes = model.make_nodes(name="FNO", jit=False) + [darcy_node]

### 初始化域，添加约束和验证器

最后，我们可以像 FNO 示例一样为模型添加约束。同样的 `SupervisedGridConstraint` 可以用来包含 PDE 损失项，我们需要为上面定义的 `darcy` 输出变量定义额外的目标值（零值，以最小化 PDE 残差），并将其添加到 `outvar_train` 字典中。

In [ ]:
# make domain
domain = Domain()

# add additional constraining values for darcy variable
outvar_train["darcy"] = np.zeros_like(outvar_train["sol"])

train_dataset = DictGridDataset(invar_train, outvar_train)
test_dataset = DictGridDataset(invar_test, outvar_test)

# add constraints to domain
supervised = SupervisedGridConstraint(
    nodes=nodes,
    dataset=train_dataset,
    batch_size=cfg.batch_size.grid,
)
domain.add_constraint(supervised, "supervised")

# add validator
val = GridValidator(
    nodes,
    dataset=test_dataset,
    batch_size=cfg.batch_size.validation,
    plotter=GridValidatorPlotter(n_examples=5),
    requires_grad=True,
)
domain.add_validator(val, "test")

### 求解器和模型训练

设置好域和配置后，即可定义求解器，并像之前一样开始训练。

In [ ]:
# make solver
slv = Solver(cfg, domain)

# start solver
slv.solve()

### 结果与后处理

网络目录文件夹包含多个不同验证预测的图表。其中一个如下所示。

<figure>
    <figcaption>PINO 验证预测。从左到右为输入渗透率及其空间导数、真实压力、预测压力、误差。</figcaption>
    <img src="pino_darcy_pred.png" alt="Drawing" style="width: 900px;"/>
</figure>

### 与 FNO 的比较

下方的 Tensorboard 图比较了 PINO（所有三种梯度方法）和 FNO 的验证损失。您可以看到，在大量训练数据（1000 个训练样本）的情况下，FNO 和 PINO 的表现相似。

<img src="pino_darcy_tensorboard1.png" alt="Drawing" style="width: 600px;"/>

PINO 的一个优势在于 PDE 损失会正则化模型，这意味着它在“小数据”环境下效率更高。下图显示了当两个模型都仅使用 100 个训练样本进行训练时的验证损失。在这种情况下，我们发现 PINO 的表现优于 FNO。

<img src="pino_darcy_tensorboard2.png" alt="Drawing" style="width: 600px;"/>

## 下一步

在本课程的最后一部分，你将进行一个小练习，使用 PINN 方法求解纳维-斯托克斯方程的流体力学问题。

请继续阅读[下一个笔记本](../chip_2d/Exercise_CFD_Problem_Notebook.ipynb)。